In [1]:
import os

In [2]:
%pwd

'c:\\Users\\artit\\IKP_2025\\fraud_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\artit\\IKP_2025\\fraud_prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class PrepareBaseModelConfig: 
    root_dir : Path 
    base_model_path : Path
    update_base_model_path : Path
    training_data: Path
    params_num_features : list 
    params_learning_rate : float
    params_include_top: bool
    params_weights: str
    params_classes : int

In [6]:
from fraud_prediction.constants import * 
from fraud_prediction.utils.common import read_yaml, create_directories
from fraud_prediction.entity.config_entity import TrainingConfig
import tensorflow as tf 

[2026-01-26 14:48:37,405: INFO: utils: Note: NumExpr detected 22 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.]
[2026-01-26 14:48:37,406: INFO: utils: NumExpr defaulting to 16 threads.]


In [7]:
import tensorflow as tf 
import numpy as np

tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

os.environ["KERAS_BACKEND"]="tensorflow"

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training_config = self.config.training
        prepare_base_model = self.config.prepare_base_model 
        params = self.params 

        training_data = self.config.data_ingestion.unzip_dir
        
        create_directories([
            Path(training_config.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir = Path(training_config.root_dir),
            trained_model_path = Path(training_config.trained_model_path),
            update_base_model_path=Path(prepare_base_model.update_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_num_features = params.NUM_FEATURES,
            params_sampling_ratio=params.SAMPLING_RATIO
        )

        return training_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
import tensorflow as tf 
import os
import glob

class Training: 
    def __init__(self, config: TrainingConfig):
        self.config = config
        
    def get_base_model(self):
        """โหลด model ANN ที่เตรียมไว้จาก stage_02"""
        self.model = tf.keras.models.load_model(
            self.config.update_base_model_path
        )

        # Re-compile immediatly with new optimizer 
        self.model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        print("Model Downloaded and Re-Compile already and ready for new training")

    def prepare_data(self):
        """เตรียมข้อมูลตามลอจิก Notebook"""
        # 1. โหลดไฟล์ CSV
        data_dir = self.config.training_data
        csv_files = glob.glob(os.path.join(data_dir, "**/*.csv"), recursive=True)
        df = pd.read_csv(csv_files[0])

        fraud_df = df[df['isFraud'] == 1]
        normal_df = df[df['isFraud'] == 0]
        
        ratio = self.config.params_sampling_ratio
        n_normal=len(fraud_df)*ratio 

        # ตรวจสอบว่ามีข้อมูลปกติพอให้สุ่มไหม
        n_normal = min(n_normal, len(normal_df)) 
        
        normal_downsampled = normal_df.sample(n=n_normal, random_state=42)
        
        df = pd.concat([fraud_df, normal_downsampled])
        df = df.sample(frac=1, random_state=42)

        print(f"ข้อมูลที่ใช้เทรน (Ratio 1:{ratio}): ทั้งหมด {df.shape[0]} แถว")

        # 2. Feature Engineering
        df['diff_new_old_balance'] = df['newbalanceOrig'] - df['oldbalanceOrg']
        df['diff_new_old_destiny'] = df['newbalanceDest'] - df['oldbalanceDest']

        # 3. Feature Selection & One-Hot Encoding
        cols_to_drop = ['nameOrig', 'nameDest', 'isFlaggedFraud','step_weeks', 'step_days'] 
        df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')
        df = pd.get_dummies(df, columns=['type'], dtype=int)
        df = df.dropna()

        # 4. แยก Feature และ Target (isFraud)
        target_col = 'isFraud'
        X = df.drop(columns=[target_col])
        y = df[target_col]

        # 5. Seperate Data and Scaling
        X_train_raw, X_valid_raw, y_train_raw, y_valid_raw = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_raw)
        X_valid_scaled = scaler.transform(X_valid_raw)

        # 6. บันทึกผลลัพธ์ลงใน Class Attribute (แปลงเป็น float32 ทันที)
        self.X_train = np.asarray(X_train_scaled).astype('float32')
        self.X_valid = np.asarray(X_valid_scaled).astype('float32')
        self.y_train = np.asarray(y_train_raw).astype('float32')
        self.y_valid = np.asarray(y_valid_raw).astype('float32')
        
        print(f" Prepared Data Done and จำนวน Features สุดท้าย: {self.X_train.shape[1]}")

    def train(self):
        """เริ่มเทรนโมเดล ANN (เวอร์ชันสมบูรณ์สำหรับข้อมูลตาราง)"""
        # 1. แปลงเป็น Tensor เพื่อประสิทธิภาพและความเสถียรบน TensorFlow
        X_train_tensor = tf.convert_to_tensor(self.X_train, dtype=tf.float32)
        y_train_tensor = tf.convert_to_tensor(self.y_train, dtype=tf.float32)
        X_valid_tensor = tf.convert_to_tensor(self.X_valid, dtype=tf.float32)
        y_valid_tensor = tf.convert_to_tensor(self.y_valid, dtype=tf.float32)

        print(f"🚀 เริ่มต้นการเทรนด้วยข้อมูล Shape: {X_train_tensor.shape}")

        # 2. Start to train 
        self.history = self.model.fit(
            X_train_tensor,
            y_train_tensor,
            epochs=self.config.params_epochs,
            batch_size=self.config.params_batch_size,
            validation_data=(X_valid_tensor, y_valid_tensor),
            verbose=1
        )

        # 3. Save model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        path.parent.mkdir(parents=True, exist_ok=True)
        model.save(str(path))

In [17]:
os.environ["KERAS_BACKEND"] = "tensorflow"
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()



In [18]:
try: 
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)

    training.get_base_model()
    training.prepare_data()

    print(f"Start to training data {training.X_train.shape[0]} transactions")
    training.train()
    print("Training completed successfully.")

except Exception as e:
    raise e

[2026-01-26 14:50:33,753: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-26 14:50:33,769: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-26 14:50:33,772: INFO: common: created directory at: artifacts]
[2026-01-26 14:50:33,773: INFO: common: created directory at: artifacts\training]
[2026-01-26 14:50:34,001: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model Downloaded and Re-Compile already and ready for new training
ข้อมูลที่ใช้เทรน (Ratio 1:50): ทั้งหมด 41871 แถว
 Prepared Data Done and จำนวน Features สุดท้าย: 13
Start to training data 33496 transactions
🚀 เริ่มต้นการเทรนด้วยข้อมูล Shape: (33496, 13)
Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9412 - loss: 0.4507 - val_accuracy: 0.9804 - val_loss: 0.2359
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.9804 - loss: 0.1433 - v